In [30]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

In [38]:
path = 'C:/Users/sache/OneDrive/Desktop/UNIVERSITY/YEAR 3 SEMESTER 1/Final_Year_Project/PlantVillage'
classes = os.listdir(path)
num_classes = len(classes)
print("no. of classes:", num_classes)
print(classes)

class_counts = {}
dataset_path = path
class_folders = os.listdir(dataset_path)
for class_name in class_folders:
    class_folder_path = os.path.join(dataset_path, class_name)
    class_counts[class_name] = len(os.listdir(class_folder_path))

# Print the number of images per class
for class_name, count in class_counts.items():
    print(f"Class {class_name}: {count} images")

no. of classes: 16
['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'PlantVillage', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_healthy', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato__Tomato_YellowLeaf__Curl_Virus']
Class Pepper__bell___Bacterial_spot: 997 images
Class Pepper__bell___healthy: 1478 images
Class PlantVillage: 15 images
Class Potato___Early_blight: 1000 images
Class Potato___healthy: 152 images
Class Potato___Late_blight: 1000 images
Class Tomato_Bacterial_spot: 2127 images
Class Tomato_Early_blight: 1000 images
Class Tomato_healthy: 1591 images
Class Tomato_Late_blight: 1909 images
Class Tomato_Leaf_Mold: 952 images
Class Tomato_Septoria_leaf_spot: 1771 images
Class Tomato_Spider_mites_Two_spotted_spider_mite: 1676 images
Class Tomato__Target_Spot:

In [ ]:
import shutil
from sklearn.model_selection import train_test_split

dataset_path = path
output_path = '/kaggle/working/split1'

classes = os.listdir(path)

# Parameters
test_size = 0.2  # 20% for test set
val_size = 0.2  # 20% of the remaining 80% for the validation set

# Function to copy files
def copy_files(files, source, dest):
    os.makedirs(dest, exist_ok=True)
    for file in files:
        file_path = os.path.join(source, file)
        if os.path.isfile(file_path):  # Ensure it's a file, not a directory
            shutil.copy(file_path, dest)

# Splitting and copying the dataset
for class_name in classes:
    class_path = os.path.join(dataset_path, class_name)
    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

    # Splitting
    train_val, test = train_test_split(images, test_size=test_size, random_state=42)
    train, val = train_test_split(train_val, test_size=val_size, random_state=42)

    # Copying files
    for dataset_type, dataset_files in zip(['train', 'val', 'test'], [train, val, test]):
        dest_path = os.path.join(output_path, dataset_type, class_name)
        copy_files(dataset_files, class_path, dest_path)

print("Dataset successfully split into training, validation, and test sets.")

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/plantdisease/PlantVillage\\Pepper__bell___Bacterial_spot'

In [ ]:
test_size1 = 0.2
val_size1 = 0.2

#path of the file

train_val,test = train_test_split(path, test_size = test_size1,random_state =42)
train,val = train_test_split(train_val, test_size = val_size1, random_state =42)
print(len(train_val))
print(len(val))
print(len(train))

73
15
58


In [32]:
train_data_gen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest',
)

val_data_gen = ImageDataGenerator(rescale = 1./255)

#flow the data in batches
train_generator = train_data_gen.flow_from_directory(
    path,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

val_generator = val_data_gen.flow_from_directory(
    path,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 41276 images belonging to 16 classes.
Found 41276 images belonging to 16 classes.


In [33]:
num_classes = 16

model = Sequential()

#first convulutional layer
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(224,224, 3)))
model.add(MaxPooling2D(2,2))

#second layer
model.add(Conv2D(64 , (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))

#third layer
model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))

#fourth layer
model.add(Conv2D(256, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))

#flattening
model.add(Flatten())

#fully connected layer
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))

#dropout
model.add(Dropout(0.3))
#output layer
model.add(Dense(num_classes, activation='softmax'))

#compile
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics= ['accuracy'])
model.summary()

c:\Users\sache\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 256)            │     9,437,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 16)             │         4,112 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,895,760 (37.75 MB)

 Trainable params: 9,895,760 (37.75 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
epoch = 5

newCNN = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    epochs = epoch,
    validation_data = val_generator,
    validation_steps = val_generator.samples // 32


)

c:\Users\sache\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
 406/1289 ━━━━━━━━━━━━━━━━━━━━ 12:07 824ms/step - accuracy: 0.4966 - loss: 2.0373

KeyboardInterrupt: 